# Import libraries

In [1]:
from os import path
import logging
from datetime import datetime
from functools import reduce
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

import importlib
import modelScripts
importlib.reload(modelScripts)
from modelScripts import buildCategoricalDf
from modelScripts import detectCategoricalColumns
from modelScripts import generateCategories
from modelScripts import createScaledArray
from modelScripts import ordered_join
from modelScripts import daysSince
from modelScripts import daysSinceDf
from modelScripts import buildPivotColumn
from modelScripts import split
from modelScripts import dropLowInfo
from modelScripts import concatDf
from modelScripts import getSparseCases


class Init_Logger(object):

    def __init__(self, logger_name, filemode='w'):
        fn = logger_name + '.log'
        logging.basicConfig(filename=path.expandvars(fn), filemode=filemode,
                            level=logging.INFO)

    def info(self, message):
        logging.info(' {dt}: {msg}'.format(msg=message,
                                           dt=datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')))


DATA_DIR = 'data'
FILES_DICT = {'users': 'users.csv', 'accounts': 'accounts.csv', 'events': 'events.csv',
              'subscriptions': 'subscriptions.csv'}

DF_DICT = {}

pd.set_option('display.max_columns', 500)
logger = Init_Logger('mondatDataPrep')
# Sample size to take
SAMPLE = 1
# In case the proportion of distinct counts out of total cases below that value - make categorial
CAT_THRESHOLD = 0.005
# same as CAT_THRESHOLD, only for accounts table
CAT_THRESHOLD_ACCOUNTS = 0.01
# For all categorial variables, replace bottom 0.5% of values with "other"
CAT_COVERAGE = 0.995
# For any categorial columns - if single value is resonsible for over 99.5% of observation - ignore it
LOW_INFO_THRESH = 0.005
# Chunk size for pivoting to save memory
pivotChunkSize = 500
ACCOUNT_ID = 'account_id'
USER_ID = 'user_id'
CAT_PREFIX = 'cat_'

In [2]:
import modelScripts
importlib.reload(modelScripts)
from modelScripts import buildCategoricalDf
from modelScripts import detectCategoricalColumns
from modelScripts import generateCategories
from modelScripts import createScaledArray
from modelScripts import ordered_join
from modelScripts import daysSince
from modelScripts import daysSinceDf
from modelScripts import buildPivotColumn
from modelScripts import split
from modelScripts import dropLowInfo
from modelScripts import concatDf
from modelScripts import getSparseCases
from modelScripts import getSparseCasesCov

# Import data

In [3]:
logger.info('Import data with sample fraction {}'.format(SAMPLE))
splAcc = pd.read_csv(path.join(DATA_DIR, 'accounts.csv'),
                     low_memory=False)[ACCOUNT_ID].sample(frac=SAMPLE).values
def importFile(moduleName, moduleFile, sample=SAMPLE):
    logger.info('Importing {}'.format(moduleName))
    df = pd.read_csv(path.join(DATA_DIR, moduleFile), low_memory=False)
    DF_DICT[moduleName] = df.loc[df[ACCOUNT_ID].isin(splAcc)].copy()
    del(df)                       
    print('module {} sample: \n'.format(moduleName))
    display(DF_DICT[moduleName].head(2))

logger.info('Importing data')
for key, val in FILES_DICT.items():
    importFile(key, val)
    
# Getting last event date - Global end date
lastDate = datetime.strptime(DF_DICT['events']['date'].max(), '%Y-%m-%d')

module users sample: 



,account_id,user_id,email,name,created_at,is_admin,pending,enabled,became_active_at,time_diff,city,region,country,serial_number,has_photo,device,os,browser,language,seniority,has_phone
0,1.0,1.0,john06@gmail.com,Jessica Ward,2018-12-31 23:58:17,1.0,0.0,1.0,2019-01-01 00:01:16,11.0,Warrawee,New South Wales,AU,1.0,1.0,desktop,windows,microsoft edge,NaN,NaN,1.0
1,2.0,2.0,doylematthew@gmail.com,Ana Spears,2018-12-31 23:57:44,1.0,0.0,1.0,2019-01-01 00:01:53,-5.0,Old Bridge,New Jersey,US,1.0,1.0,mobile,ios,NaN,NaN,NaN,1.0


module accounts sample: 



,account_id,account_name,created_at,plan_id,trial_start,churn_date,churn_reason,time_diff,region,country,subscription_started_at,paying,has_logo,device,os,browser,collection_21_days,company_size,payment_currency,max_team_size,min_team_size,industry,utm_cluster_id,mrr,user_goal,user_description,team_size,lead_score
0,1.0,"Gardner, Barron and Keller",2019-01-01 00:01:15,NaN,2019-01-01 00:01:15,NaN,NaN,11.0,New South Wales,AU,NaN,0,1,desktop,windows,microsoft edge,0,NaN,AUD,5.0,2.0,NaN,orders,NaN,NaN,NaN,NaN,0
1,2.0,Dunn Ltd,2019-01-01 00:01:52,NaN,2019-01-01 00:01:52,NaN,NaN,NaN,New Jersey,US,NaN,0,1,mobile,ios,NaN,0,NaN,USD,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0


module events sample: 



,date,user_id,account_id,total_events,column_events,board_events,num_of_boards,count_kind_columns,content_events,group_events,invite_events,import_events,notification_events,new_entry_events,payment_events,inbox_events,communicating_events,non_communicating_events,web_events,ios_events,android_events,desktop_app_events,empty_events
0,2019-01-25,217501.0,127005.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1
1,2019-01-20,82679.0,46593.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1


module subscriptions sample: 



,event_happened_at,subscription_id,account_id,plan_id,event_type,invoice_charge_amount,prev_plan_id,status,status_reason,currency,invoice_charge_amount_usd,mrr_gain,next_charge_date,payment_type,transaction_date
0,2019-01-09 22:37:05,1.0,38427.0,264.0,RECURRING,645.48,264.0,NaN,NaN,AUD,453.16,NaN,2021-01-09 22:37:00,PAYPAL,2019-01-09 22:37:00
1,2019-01-10 09:44:45,2.0,24270.0,236.0,CONTRACT_CHANGE,0.00,234.0,SUCCESS,NaN,GBP,0.00,81.0,2020-01-10 09:44:00,CC,2019-01-10 09:44:00


# Account level aggregations

## Users table

In [4]:
table = 'users'
logger.info('started working on table {}'.format(table))
df = DF_DICT[table].copy()

def positiveInd(row):
    if row['user_id'] - row['email'] > 0:
        return 1
    else:
        return 0

logger.info('Running additive aggregation on users dataset')
# define aggregation function
aggFunct = {'user_id': 'count', 'is_admin': 'sum', 'enabled': 'sum', 'has_photo': 'sum',
            'has_phone': 'sum', 'created_at': 'max', 'email': 'nunique'}

usersAgg = DF_DICT[table].groupby(ACCOUNT_ID).agg(aggFunct)
usersAgg['daysSinceLastUserCreate'] = \
    usersAgg.apply(daysSince('created_at', lastDate, dateFormat='%Y-%m-%d %H:%M:%S'), axis=1)
usersAgg.drop(columns=['created_at'], inplace=True)

usersAgg['daysSinceLastUserCreate'] = \
    pd.cut(usersAgg['daysSinceLastUserCreate'], 3, labels=["small", "medium", "large"]).astype(str)

# adding indicator for duplicate emails
usersAgg['hasDuplicateEmails'] = usersAgg.apply(positiveInd, axis=1)

logger.info('Running pivot count aggregation on {}'.format(table))
pivotColumns = ['seniority', 'country', 'device', 'os', 'browser', 'language']


# run aggregation by chunks
logger.info('Getting pivot chunks for {}'.format(table))
chunks = split(df[ACCOUNT_ID].unique(), pivotChunkSize)

aggList = []
for pc in pivotColumns:
    logger.info('substituting cases having low frequency with {dft} in {col}'.format(dft='other',
                                                                                     col=pc))
    df.fillna('nan', inplace=True)
    c_valCounts = df[pc].value_counts()
    logger.info('Old categories count for {pc} is {cnt}'.
                format(pc=pc, cnt=c_valCounts.shape[0]))
    sparseCases = getSparseCasesCov(c_valCounts, coverage=CAT_COVERAGE)
    if len(sparseCases) > 0:
            coverage = 1 - c_valCounts.loc[sparseCases].sum()/c_valCounts.sum()
            df[pc] = df[pc].replace(sparseCases, 'other')
            logger.info('New categories count for {pc} is {cnt}, coverage is {cv}'.
                        format(pc=pc, cnt=df[pc].value_counts().shape[0], cv=coverage))
            
    dataColumns = [ACCOUNT_ID, USER_ID, pc]
    dfList = []
    chunksCnt = len(chunks)
    for chunkId, chunk in enumerate(chunks):
        if chunkId%pivotChunkSize == 0:
            logger.info('Started pivoting chunk {ch} out of {tot} for column {col}'.
                        format(ch=chunkId, tot=chunksCnt, col=pc))
        data = df.loc[df[ACCOUNT_ID].isin(chunk.tolist())][dataColumns]       
        dfTmp = pd.pivot_table(data=data, index=ACCOUNT_ID, values=USER_ID, columns=pc, aggfunc='count')
        dfTmp.columns = [buildPivotColumn(colVal=col, orgCol=pc) for col in dfTmp.columns.values]
        dfList.append(dfTmp.copy())
        del(dfTmp)
        del(data)
    aggList.append(pd.concat(dfList, ignore_index=False, sort=False).fillna(0))
    del(dfList)
dfTemp = usersAgg.merge(right=ordered_join(aggList), left_index=True, right_index=True)
usersAgg, nonCatColumns = buildCategoricalDf(df=dfTemp, catThreshold=CAT_THRESHOLD, excludedColumns=[])
print('Non-categorial columns: {}'.format(nonCatColumns))
del(dfTemp)

logger.info('Dropping low info colunbs for {}'.format(table))
columnsToDrop = dropLowInfo(df=usersAgg, thresh=LOW_INFO_THRESH)
usersAgg.drop(columns=columnsToDrop, axis=1, inplace=True)
logger.info('Dropped columns {col} due to low information'.format(col=columnsToDrop))

usersAgg.columns = ['{col}_{tbl}'.format(col=col, tbl=table) for col in usersAgg.columns.values]

del(df)
usersAgg.head()

Non-categorial columns: ['user_id', 'enabled', 'has_photo', 'has_phone', 'email', 'seniority_nan', 'device_desktop', 'language_nan']
for column cat_is_admin the value is 0.11544396530054801
for column cat_daysSinceLastUserCreate the value is 0.42462141735978054
for column cat_hasDuplicateEmails the value is 0.005097110427151774
for column cat_seniority_director the value is 0.015316467716438686
for column cat_seniority_executive the value is 0.020040022790367762
for column cat_seniority_manager the value is 0.02742594526960218
for column cat_country_AE the value is 0.013431933327002654
for column cat_country_AR the value is 0.01930617853571892
for column cat_country_AT the value is 0.002827150701306569
for column cat_country_AU the value is 0.03626978097786315
for column cat_country_BE the value is 0.007186227467978923
for column cat_country_BR the value is 0.11425347581037071
for column cat_country_CA the value is 0.03150293537701865
for column cat_country_CH the value is 0.0053708182

,cat_is_admin_users,cat_daysSinceLastUserCreate_users,cat_hasDuplicateEmails_users,cat_seniority_director_users,cat_seniority_executive_users,cat_seniority_manager_users,cat_country_AE_users,cat_country_AR_users,cat_country_AU_users,cat_country_BE_users,cat_country_BR_users,cat_country_CA_users,cat_country_CH_users,cat_country_CO_users,cat_country_DE_users,cat_country_DK_users,cat_country_EC_users,cat_country_ES_users,cat_country_FR_users,cat_country_GB_users,cat_country_HK_users,cat_country_ID_users,cat_country_IL_users,cat_country_IN_users,cat_country_IT_users,cat_country_MX_users,cat_country_NZ_users,cat_country_PL_users,cat_country_PT_users,cat_country_SA_users,cat_country_SE_users,cat_country_SG_users,cat_country_US_users,cat_country_nan_users,cat_country_other_users,cat_country_CL_users,cat_country_NL_users,cat_country_PE_users,cat_country_ZA_users,cat_country_CR_users,cat_country_RO_users,cat_device_mobile_users,cat_device_nan_users,cat_device_tablet_users,cat_os_android_users,cat_os_chrome_os_users,cat_os_ios_users,cat_os_linux_users,cat_os_mac_users,cat_os_nan_users,cat_os_windows_users,cat_browser_chrome_users,cat_browser_firefox_users,cat_browser_generic_browser_users,cat_browser_internet_explorer_users,cat_browser_microsoft_edge_users,cat_browser_nan_users,cat_browser_safari_users,cat_language_de_users,cat_language_es_users,cat_language_fr_users,cat_language_pt_users,user_id_users,enabled_users,has_photo_users,has_phone_users,email_users,seniority_nan_users,device_desktop_users,language_nan_users
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,1.0,large,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1,1.0,1.0,1.0
2.0,1.0,large,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,2.0,2.0,2.0,2,2.0,0.0,2.0
3.0,1.0,large,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1,1.0,1.0,1.0
4.0,1.0,large,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1,1.0,0.0,1.0
5.0,1.0,large,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1,1.0,1.0,1.0


## Accounts table

In [5]:
table = 'accounts'
logger.info('started working on table {}'.format(table))

accounts = DF_DICT[table].groupby('account_id').size()
uniqueAccounts = accounts.loc[accounts == 1].index.tolist()

df = DF_DICT[table].loc[DF_DICT[table]['account_id'].isin(uniqueAccounts)].copy()
df.set_index(ACCOUNT_ID, inplace=True)

excludedCatColumns=['lead_score', 'churn_date', 'created_at', 'trial_start', 'mrr', 'company_size',
                    'account_name', 'region', 'subscription_started_at']
df_cat, nonCatColumns = buildCategoricalDf(df=t_t_, catThreshold=CAT_THRESHOLD_ACCOUNTS,
                                           excludedColumns=excludedCatColumns)


print('Non-categorial columns: {}'.format(nonCatColumns))
df_company_size = pd.DataFrame(pd.cut(df['company_size'], 3,
                                      labels=["small", "medium", "large"]).astype(str))
df_company_size.columns = ['cat_accounts_company_size']

df_created_at = daysSinceDf(df, columnName='created_at', lastDate=lastDate)
df['mrr'].fillna(0, inplace=True)


accountsAgg = ordered_join([df_cat.drop(excludedCatColumns, axis=1), df_company_size, df_created_at, df[['mrr']]])
accounts_y = df['lead_score'].to_frame()
accounts_y.columns = ['y']
del(df)

logger.info('Dropping low info columns for {}'.format(table))
columnsToDrop = dropLowInfo(df=accountsAgg, thresh=LOW_INFO_THRESH)
accountsAgg.drop(columns=columnsToDrop, axis=1, inplace=True)
logger.info('Dropped columns {col} due to low information'.format(col=columnsToDrop))

accountsAgg.columns = ['{col}_{tbl}'.format(col=col, tbl=table) for col in accountsAgg.columns.values]
accountsAgg.head()

Non-categorial columns: ['account_name', 'created_at', 'plan_id', 'trial_start', 'churn_date', 'region', 'country', 'subscription_started_at', 'collection_21_days', 'company_size', 'mrr', 'lead_score']
for column cat_churn_reason the value is 0.017616487067687703
for column cat_time_diff the value is 0.8311967227865522
for column cat_paying the value is 0.020987586291560723
for column cat_has_logo the value is 0.0
for column cat_device the value is 0.28846230304738163
for column cat_os the value is 0.47352818700707355
for column cat_browser the value is 0.3797930746336118
for column cat_payment_currency the value is 0.23064582393138666
for column cat_max_team_size the value is 0.6790027182233965
for column cat_min_team_size the value is 0.6789971380881201
for column cat_industry the value is 0.6393991031327577
for column cat_utm_cluster_id the value is 0.4858086697166756
for column cat_user_goal the value is 0.35618142972831024
for column cat_user_description the value is 0.22688272020

,cat_churn_reason_accounts,cat_time_diff_accounts,cat_paying_accounts,cat_device_accounts,cat_os_accounts,cat_browser_accounts,cat_payment_currency_accounts,cat_max_team_size_accounts,cat_min_team_size_accounts,cat_industry_accounts,cat_utm_cluster_id_accounts,cat_user_goal_accounts,cat_user_description_accounts,cat_team_size_accounts,plan_id_accounts,country_accounts,collection_21_days_accounts,cat_accounts_company_size_accounts,daysSince_created_at_accounts,mrr_accounts
account_id,,,,,,,,,,,,,,,,,,,,
1.0,nan,11.0,0,desktop,windows,microsoft edge,AUD,5.0,2.0,nan,orders,nan,nan,nan,NaN,AU,0,nan,193,0.0
2.0,nan,nan,0,mobile,ios,nan,USD,5.0,2.0,nan,nan,nan,nan,nan,NaN,US,0,nan,193,0.0
3.0,nan,-6.0,0,desktop,windows,chrome,USD,1.0,1.0,Other,todos,nan,nan,1,NaN,US,0,nan,193,0.0
4.0,nan,nan,0,mobile,android,nan,USD,nan,nan,nan,nan,nan,nan,nan,NaN,IL,0,nan,193,0.0
5.0,nan,-5.0,0,desktop,chrome_os,chrome,USD,1.0,1.0,Design,todos,nan,nan,1,NaN,US,0,nan,193,0.0


## Events table

In [6]:
table = 'events'
logger.info('started working on table {}'.format(table))

df = DF_DICT[table].copy()

# last and first events
dfMax = df.groupby(ACCOUNT_ID).agg({'date': ['max', 'min']})
dfMax.columns = ['lastEvent', 'firstEvent']
dfMax['daysSincelastEvent'] = dfMax.apply(daysSince('lastEvent', lastDate,
                                                    dateFormat='%Y-%m-%d'), axis=1)
dfMax['daysSincefirstEvent'] = dfMax.apply(daysSince('firstEvent', lastDate,
                                                     dateFormat='%Y-%m-%d'), axis=1)

dfMax['daysSincelastEvent'] = \
    pd.cut(dfMax['daysSincelastEvent'], 3, labels=["small", "medium", "large"]).astype(str)

dfMax['daysSincefirstEvent'] = \
    pd.cut(dfMax['daysSincefirstEvent'], 3, labels=["small", "medium", "large"]).astype(str)

dfMax.drop(columns=['lastEvent', 'firstEvent'], axis=1, inplace=True)

CAT_EXCLUDED_COLUMNS = []

# sum up all additive events
sumColumns = [c for c in df.columns.values if c.endswith('events')]
dfSum = df.groupby(ACCOUNT_ID)[sumColumns].sum()
dfSum.columns = ['{col}_sum'.format(col=col) for col in dfSum.columns.values]

# count user-days
cntColumns = ['user_id']
dfCnt = df.groupby(ACCOUNT_ID)[cntColumns].count()
dfCnt.columns = ['{col}_cnt'.format(col=col) for col in dfCnt.columns.values]

# for each additive metric count distinct users
uniqueDfList = []
for c in sumColumns:
    data = df.loc[df[c] > 0].groupby(ACCOUNT_ID)[USER_ID].nunique().to_frame()
    data.columns = ['unique_users_{}'.format(c)]       
    uniqueDfList.append(data)

# Merging  main tables 
eventsMrg = ordered_join([dfCnt, dfSum, dfMax])
del([dfCnt, dfSum, dfMax])

# adding distinct count frames
for d in uniqueDfList:
    col = d.columns.values[0]
    eventsMrg = eventsMrg.merge(right=d, left_index=True, right_index=True, how='left')
    eventsMrg[col].fillna(0, inplace=True)

eventsAgg, nonCatColumns = buildCategoricalDf(df=eventsMrg, catThreshold=CAT_THRESHOLD,
                                              excludedColumns=CAT_EXCLUDED_COLUMNS)

logger.info('Dropping low info colunbs for {}'.format(table))
columnsToDrop = dropLowInfo(df=eventsAgg, thresh=LOW_INFO_THRESH)
eventsAgg.drop(columns=columnsToDrop, axis=1, inplace=True)
logger.info('Dropped columns {col} due to low information'.format(col=columnsToDrop))

eventsAgg.columns = ['{col}_{tbl}'.format(col=col, tbl=table) for col in eventsAgg.columns.values]
print('Non-categorial columns: {}'.format(nonCatColumns))
del(df)
display(eventsAgg.head())

for column cat_daysSincelastEvent the value is 0.6501655714622618
for column cat_daysSincefirstEvent the value is 0.6149811124336981
for column cat_unique_users_column_events the value is 0.31709063101356527
for column cat_unique_users_board_events the value is 0.4276892977900082
for column cat_unique_users_content_events the value is 0.3814911618469765
for column cat_unique_users_group_events the value is 0.1242518584053508
for column cat_unique_users_invite_events the value is 0.17810200541718013
for column cat_unique_users_import_events the value is 0.07765880696446736
for column cat_unique_users_notification_events the value is 0.0376488433720481
for column cat_unique_users_new_entry_events the value is 0.26028978145683346
for column cat_unique_users_payment_events the value is 0.1864866335756319
for column cat_unique_users_inbox_events the value is 0.3430308578318162
for column cat_unique_users_communicating_events the value is 0.443812455606893
for column cat_unique_users_non_com

,cat_daysSincelastEvent_events,cat_daysSincefirstEvent_events,cat_unique_users_column_events_events,cat_unique_users_board_events_events,cat_unique_users_content_events_events,cat_unique_users_group_events_events,cat_unique_users_invite_events_events,cat_unique_users_import_events_events,cat_unique_users_notification_events_events,cat_unique_users_new_entry_events_events,cat_unique_users_payment_events_events,cat_unique_users_inbox_events_events,cat_unique_users_communicating_events_events,cat_unique_users_non_communicating_events_events,cat_unique_users_web_events_events,cat_unique_users_ios_events_events,cat_unique_users_android_events_events,cat_unique_users_desktop_app_events_events,user_id_cnt_events,total_events_sum_events,column_events_sum_events,board_events_sum_events,content_events_sum_events,group_events_sum_events,invite_events_sum_events,import_events_sum_events,notification_events_sum_events,new_entry_events_sum_events,payment_events_sum_events,inbox_events_sum_events,communicating_events_sum_events,non_communicating_events_sum_events,web_events_sum_events,ios_events_sum_events,android_events_sum_events,desktop_app_events_sum_events,empty_events_sum_events,unique_users_total_events_events,unique_users_empty_events_events
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,large,large,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,6,53.0,0,0,0,0,0,1,0,1,0,1,0,2.0,33,0,0,0,7,1,1.0
2.0,large,large,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,12,1428.0,0,0,0,0,0,0,6,27,0,0,7,27.0,0,1138,183,0,8,3,3.0
3.0,large,large,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,6,72.0,0,1,1,0,0,2,0,1,0,1,1,5.0,52,0,0,0,9,1,1.0
4.0,large,large,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,7,97.0,0,0,0,0,0,0,0,2,0,0,0,2.0,0,0,42,0,7,1,1.0
5.0,large,large,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,6,158.0,7,2,5,1,0,2,0,1,0,4,0,23.0,139,0,0,0,9,1,1.0


## Subscriptions table

In [7]:
table = 'subscriptions'
logger.info('started working on table {}'.format(table))
EXCLUDED_COLUMNS = []

df = DF_DICT[table].copy()

df['plan_id'] = df['plan_id'].astype(str)
df['prev_plan_id'] = df['prev_plan_id'].astype(str)
df['status'].fillna('missing', inplace=True)
df['event_type_status'] = df.apply(concatDf(['event_type', 'status']), axis=1)

pivotColumns = ['currency', 'payment_type', 'event_type_status']

# define aggregation function
aggFunct = {'plan_id': 'count', 'mrr_gain': 'sum', 'event_happened_at': 'max'}

subsAgg = df.groupby(ACCOUNT_ID).agg(aggFunct)
subsAgg.columns = ['subsCnt', 'mrr_gain', 'lastEventHappenedAt']
subsAgg['daysSincelastEventHappenedAt'] = \
    subsAgg.apply(daysSince('lastEventHappenedAt', lastDate, dateFormat='%Y-%m-%d %H:%M:%S'), axis=1)
subsAgg.drop(columns=['lastEventHappenedAt'], axis=1, inplace=True)

subsAgg['daysSincelastEventHappenedAt'] = \
    pd.cut(subsAgg['daysSincelastEventHappenedAt'], 3, labels=["small", "medium", "large"]).astype(str)

logger.info('Getting pivot chunks for {}'.format(table))
chunks = split(df[ACCOUNT_ID].unique(), pivotChunkSize)

aggList = []
chunksCnt = len(chunks)
for chunkId, pc in enumerate(pivotColumns):
    
    df.fillna('nan', inplace=True)
    c_valCounts = df[pc].value_counts()
    t_cases = c_valCounts.sum()
    sparseCases = getSparseCasesCov(c_valCounts, coverage=CAT_COVERAGE)
    if len(sparseCases) > 0:
            df[pc] = df[pc].replace(sparseCases, 'other')
            logger.info('New categories count for {pc} is {cnt}'.
            format(pc=pc, cnt=df[pc].value_counts().shape[0]))
            
    dataColumns = [ACCOUNT_ID, pc, 'event_happened_at']
    dfList = []
    for chunk in chunks:
        if chunkId%pivotChunkSize == 0:
            logger.info('Started pivoting chunk {ch} out of {tot} for column {col}'.
                        format(ch=chunkId, tot=chunksCnt, col=pc))
        data = df.loc[df[ACCOUNT_ID].isin(chunk.tolist())][dataColumns]        
        dfTmp = pd.pivot_table(data=data, index=ACCOUNT_ID, values='event_happened_at',
                               columns=pc, aggfunc='count')
        dfTmp.columns = [buildPivotColumn(colVal=col, orgCol=pc) for col in dfTmp.columns.values]
        dfList.append(dfTmp.copy())
        del(dfTmp)
        del(data)
    aggList.append(pd.concat(dfList, ignore_index=False, sort=False).fillna(0))
    del(dfList)
dfPiv = subsAgg.merge(right=ordered_join(aggList), left_index=True, right_index=True)
subsAgg, nonCatColumns = buildCategoricalDf(df=dfPiv, catThreshold=CAT_THRESHOLD,
                                             excludedColumns=EXCLUDED_COLUMNS)
del([dfPiv, df])
print('Non-categorial columns: {}'.format(nonCatColumns))

logger.info('Dropping low info colunbs for {}'.format(table))
columnsToDrop = dropLowInfo(df=subsAgg, thresh=LOW_INFO_THRESH)
subsAgg.drop(columns=columnsToDrop, axis=1, inplace=True)
logger.info('Dropped columns {col} due to low information'.format(col=columnsToDrop))

subsAgg.columns = ['{col}_{tbl}'.format(col=col, tbl=table) for col in subsAgg.columns.values]
display(subsAgg.head())

Non-categorial columns: ['mrr_gain']
for column cat_subsCnt the value is 0.641383918196607
for column cat_daysSincelastEventHappenedAt the value is 0.372850336974204
for column cat_currency_AUD the value is 0.05812805019753664
for column cat_currency_CAD the value is 0.04052405298628858
for column cat_currency_EUR the value is 0.09975598419707177
for column cat_currency_GBP the value is 0.07175226586102723
for column cat_currency_USD the value is 0.7266732512200791
for column cat_currency_nan the value is 0.11712758540553103
for column cat_currency_other the value is 0.0038926330467116133
for column cat_payment_type_BALANCE the value is 0.04209272600511271
for column cat_payment_type_CC the value is 0.6649721124796654
for column cat_payment_type_PAYPAL the value is 0.10910992330931912
for column cat_payment_type_nan the value is 0.10591447827097378
for column cat_payment_type_other the value is 0.00917964211015565
for column cat_event_type_status_CANCELLATION_missing the value is 0.121

,cat_subsCnt_subscriptions,cat_daysSincelastEventHappenedAt_subscriptions,cat_currency_AUD_subscriptions,cat_currency_CAD_subscriptions,cat_currency_EUR_subscriptions,cat_currency_GBP_subscriptions,cat_currency_USD_subscriptions,cat_currency_nan_subscriptions,cat_payment_type_BALANCE_subscriptions,cat_payment_type_CC_subscriptions,cat_payment_type_PAYPAL_subscriptions,cat_payment_type_nan_subscriptions,cat_payment_type_other_subscriptions,cat_event_type_status_CANCELLATION_missing_subscriptions,cat_event_type_status_CANCEL_ON_RENEWAL_missing_subscriptions,cat_event_type_status_CC_CHARGE_FAILED_missing_subscriptions,cat_event_type_status_CHARGE_missing_subscriptions,cat_event_type_status_CONTRACT_CHANGE_FAILED_subscriptions,cat_event_type_status_CONTRACT_CHANGE_SUCCESS_subscriptions,cat_event_type_status_FREE_DAYS_WERE_GIVEN_missing_subscriptions,cat_event_type_status_NEW_CONTRACT_SIGNED_missing_subscriptions,cat_event_type_status_PAYMENT_METHOD_UPDATED_missing_subscriptions,cat_event_type_status_PLAN_ENFORCER_UPGRADE_missing_subscriptions,cat_event_type_status_RECURRING_missing_subscriptions,cat_event_type_status_REFUND_missing_subscriptions,cat_event_type_status_SUBSCRIPTION_CHARGE_FAILURE_missing_subscriptions,cat_event_type_status_USER_ADDED_missing_subscriptions,cat_event_type_status_USER_DELETED_missing_subscriptions,cat_event_type_status_other_subscriptions,cat_event_type_status_ONETIME_CHARGE_missing_subscriptions,mrr_gain_subscriptions
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13.0,3,large,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72.0,7,medium,0.0,5.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0
209.0,1,large,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0
235.0,3,small,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0
294.0,1,large,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0


## Merging the tables

In [8]:
logger.info('Merging accounts')
dfTotal = accountsAgg.copy()
del(accountsAgg)
logger.info('Merging users')
dfTotal = dfTotal.merge(right=usersAgg, right_index=True, left_index=True, how='left')
del(usersAgg)
logger.info('Merging events')
dfTotal = dfTotal.merge(right=eventsAgg, right_index=True, left_index=True, how='left')
del(eventsAgg)
logger.info('Merging subscriptions')
dfTotal = dfTotal.merge(right=subsAgg, right_index=True, left_index=True, how='left')
del(subsAgg)
logger.info('Merging target')
dfTotal = dfTotal.merge(right=accounts_y, right_index=True, left_index=True, how='left')
del(DF_DICT)

for c in dfTotal.columns.values.tolist():
    if c.startswith('cat_'):
        dfTotal[c].fillna('nan', inplace=True)
        c_valCounts = dfTotal[c].value_counts()
        logger.info('Old categories count for {pc} is {cnt}'.
                     format(pc=c, cnt=c_valCounts.shape[0]))
        sparseCases = getSparseCasesCov(c_valCounts, coverage=CAT_COVERAGE)
        if len(sparseCases) > 0:
            coverage = 1 - c_valCounts.loc[sparseCases].sum()/c_valCounts.sum()
            dfTotal[c] = dfTotal[c].replace(sparseCases, 'other')
            logger.info('New categories count for {pc} is {cnt}, coverage is {cv}'.
                        format(pc=c, cnt=dfTotal[c].value_counts().shape[0], cv=coverage))
        
    else:
        dfTotal[c].fillna(0, inplace=True)

columnsToDrop = dropLowInfo(df=dfTotal, thresh=LOW_INFO_THRESH)
dfTotal.drop(columns=columnsToDrop, axis=1, inplace=True)

dfTotal.to_csv(path.join(DATA_DIR, 'dfTotal.csv'))

for column cat_churn_reason_accounts the value is 0.017616487067687703
for column cat_time_diff_accounts the value is 0.8311967227865522
for column cat_paying_accounts the value is 0.020987586291560723
for column cat_device_accounts the value is 0.28846230304738163
for column cat_os_accounts the value is 0.47352818700707355
for column cat_browser_accounts the value is 0.3797930746336118
for column cat_payment_currency_accounts the value is 0.23064582393138666
for column cat_max_team_size_accounts the value is 0.6790027182233965
for column cat_min_team_size_accounts the value is 0.6789971380881201
for column cat_industry_accounts the value is 0.6393991031327577
for column cat_utm_cluster_id_accounts the value is 0.4858086697166756
for column cat_user_goal_accounts the value is 0.35618142972831024
for column cat_user_description_accounts the value is 0.22688272020434452
for column cat_team_size_accounts the value is 0.7010282096763731
for column cat_accounts_company_size_accounts the val

for column cat_event_type_status_NEW_CONTRACT_SIGNED_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_PAYMENT_METHOD_UPDATED_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_PLAN_ENFORCER_UPGRADE_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_RECURRING_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_REFUND_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_SUBSCRIPTION_CHARGE_FAILURE_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_USER_ADDED_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_USER_DELETED_missing_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_other_subscriptions the value is 0.024011322094475873
for column cat_event_type_status_ONETIME_CHARGE_missing_subscriptio

In [39]:
# sanity check:
catSizes = []
numCorrect = []
for c in dfTotal.columns.values:
    if c.startswith(CAT_PREFIX):
        catSizes.append([c, dfTotal[c].value_counts().shape[0]])
    else:
        if dfTotal[c].dtype in ['float64', 'int64']:
            numCorrect.append([c, 1])
        else:
            numCorrect.append([c, 0])

In [40]:
pd.DataFrame(catSizes).sort_values(by=1, ascending=False)

,0,1
14,cat_country_accounts,101
9,cat_industry_accounts,31
1,cat_time_diff_accounts,25
10,cat_utm_cluster_id_accounts,15
8,cat_min_team_size_accounts,12
67,cat_browser_chrome_users,11
13,cat_team_size_accounts,11
7,cat_max_team_size_accounts,11
66,cat_os_windows_users,10
91,cat_unique_users_non_communicating_events_events,9


In [27]:
dfTotal.rename(columns={'country_accounts': CAT_PREFIX+'country_accounts'}, inplace=True)

In [38]:
c_valCounts = dfTotal['cat_country_accounts'].value_counts()
t_cases = c_valCounts.sum()
sparseCases = getSparseCasesCov(c_valCounts, coverage=CAT_COVERAGE)
dfTotal['cat_country_accounts'] = dfTotal['cat_country_accounts'].replace(sparseCases, 'other')

In [41]:
dfTotal.to_csv(path.join(DATA_DIR, 'dfTotal.csv'))